In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from tensorboard.backend.event_processing import event_accumulator
import pickle

In [2]:
def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    # assert all(
    #     s in ea.Tags()["scalars"] for s in scalars
    # ), "some scalars were not found in the event accumulator"
    res = {}
    for s in scalars:
        try:
            res[s] = pd.DataFrame(ea.Scalars(s))
        except:
            continue
    return res

In [3]:
scalars = [
    "val/acc_adv_attack",
    "val/balanced_acc_adv_attack",
    "val/acc_task",
    "val/balanced_acc_task",
    "train/zero_ratio_adv",
    "train/zero_ratio_task",
    "val/acc_task_debiased",
    "val/acc_protected",
    "val/balanced_acc_task_debiased",
    "val/balanced_acc_protected"
]
# experiment_names = [
#     'task_baseline_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'adverserial_baseline_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'task_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'task_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'adverserial_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'adverserial_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'modular_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05',
#     'modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05'
# ]
experiment_names = [
    "only_adv_attack_bert_uncased_L-4_H-256_A-4-fixmask0.1-modular-sparse_task_64_0.0001_seed{}",
    "only_adv_attack_bert_uncased_L-4_H-256_A-4-fixmask0.05-modular-sparse_task_64_0.0001_seed{}"
]

In [4]:
#folder = "/share/home/lukash/logs_bert_L4/seed{}"
# results = {}
# for n in experiment_names:
#     results[n] = []
#     for i in range(5):
#         filepath = os.path.join(folder.format(i), n)
#         df = parse_tensorboard(filepath, scalars)
#         results[n].append(df)

In [ ]:
folder = "../logs_bios"
results = {}
for n in experiment_names:
    results[n] = []
    for i in range(5):
        filepath = os.path.join(folder, n.format(i))
        df = parse_tensorboard(filepath, scalars)
        results[n].append(df)

In [5]:
with open("results.pkl", "wb") as f:
    pickle.dump(results, f)

In [9]:
[x["val/acc_adv_attack"].tail(1) for x in results['modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05']]

[       wall_time  step    value
 39  1.655004e+09    39  0.52642,
        wall_time  step     value
 39  1.655949e+09    39  0.567327,
        wall_time  step     value
 39  1.655949e+09    39  0.554078,
        wall_time  step     value
 39  1.655950e+09    39  0.533058,
        wall_time  step     value
 39  1.655950e+09    39  0.546913]

In [12]:
test = [x["val/acc_adv_attack"]["value"].max() for x in results['modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05']]
sum(test)/len(test)

0.5712411761283874

In [5]:
scalar_stats = {}
for s in scalars:
    scalar_stats[s] = {}
    for k,v in results.items():
            try:
                scalar = np.array([x[s]["value"].max() for x in v])
                scalar_stats[s][k] = [scalar.mean(), scalar.std()]
            except KeyError:
                continue
        

In [14]:
with open("stats.pkl", "wb") as f:
    pickle.dump(scalar_stats, f)

In [2]:
with open("stats.pkl", "rb") as f:
    scalar_stats = pickle.load(f)

In [3]:
scalar_stats.keys()

dict_keys(['val/acc_adv_attack', 'val/balanced_acc_adv_attack', 'val/acc_task', 'val/balanced_acc_task', 'train/zero_ratio_adv', 'train/zero_ratio_task', 'val/acc_task_debiased', 'val/acc_protected', 'val/balanced_acc_task_debiased', 'val/balanced_acc_protected'])

In [6]:
scalar_stats['val/acc_task_debiased']

{'adverserial_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8344484329223633,
  0.0005988100637620501],
 'adverserial_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8346749663352966,
  0.00026737418148785764],
 'modular_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8346486210823059,
  0.0006845799638344238],
 'modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8343114614486694,
  0.0005864827759984568]}

In [7]:
scalar_stats['val/acc_task']

{'task_baseline_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8317353248596191,
  0.0008140872150845018],
 'task_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.835544204711914,
  0.0006938230235159399],
 'task_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8357496619224548,
  0.0008905420053891446],
 'modular_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8351648807525635,
  0.0005305981033382874],
 'modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8353387475013733,
  0.0004914859291633344]}

In [70]:
scalar_stats['val/acc_task_debiased']

{'adverserial_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8344484329223633,
  0.0005988100637620501],
 'adverserial_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8346749663352966,
  0.00026737418148785764],
 'modular_diff_pruning_0.05_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8346486210823059,
  0.0006845799638344238],
 'modular_diff_pruning_0.1_bert_uncased_L-4_H-256_A-4_64_2e-05': [0.8343114614486694,
  0.0005864827759984568]}